# Final Project | How to predict a pandemic 
by Kevin Spurk

Text

# Table of content

1. 
2.
3.
...

# 1 | Setup

### 1.1. Libary imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import random
import datetime
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.set_option('max_row', None)

### 1.2 Data imports

### 1.2.1 google mobility data

In [5]:
# csv import
data_mob_global = pd.read_csv('data/mobility_google/Global_Mobility_Report.csv')

# dict of countries and there languages to search in 
countries_lang = {'GB': 'en', 'DE': 'de', 'FR': 'fr', 'NL': 'nl',
                  'ES': 'es', 'PT': 'pt', 'PL': 'pl', 'IT': 'it',
                  'AT': 'de', 'DK': 'da', 'SE': 'sv', 'GR': 'el', 'CZ': 'cs'}

# getting subset of row with data of the target countries
tgt_countries = [country_code for country_code in countries_lang.keys()]
data_mob_eu = data_mob_global[data_mob_global['country_region_code'].isin(tgt_countries)]

# dropping unwanted columns
data_mob_eu = data_mob_eu.drop(data_mob_eu.iloc[:, 1:8], axis=1)

len(data_mob_eu)


1363509

In [8]:
data_mob_eu.head()

,country_region_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
274011,AT,2020-02-15,9.0,1.0,42.0,13.0,0.0,-2.0
274012,AT,2020-02-16,15.0,21.0,42.0,12.0,1.0,-2.0
274013,AT,2020-02-17,9.0,5.0,35.0,3.0,-4.0,0.0
274014,AT,2020-02-18,8.0,5.0,40.0,2.0,-4.0,0.0
274015,AT,2020-02-19,4.0,2.0,10.0,-1.0,-5.0,1.0


### 1.2.2 facebook mobility data

### 1.2.3 google search trends data

In [9]:
# connect to google and deepL APIs 
from pytrends.request import TrendReq
from pytrends import dailydata
from translate import Translator 

# get authentification for deepL API
with open('auth/deepl_auth_key.txt') as f:
    deepl_auth_key = f.read()

# google trends API access request
pytrends = TrendReq(hl='en-US', tz=0)

In [10]:
# keyword list 
keywords_list = ['covid', 'corona', 'SARS-CoV-2', 'virus', 'symptoms', 'fever', 'cough', 'tiredness', 'loss of smell', 'throat', 'shortness of breath', 'breathing issues', 'headache', 'chest pain', 'corona test', 'covid test', 'test', 'sick', 'infection']

# translating keyword list into target languages
from pyf.apis import deepl_translate_list

kw_dict, kw_df = deepl_translate_list(auth=deepl_auth_key, keywords=keywords_list, lang_out=countries_lang)


In [11]:
kw_df.head()

,GB,DE,FR,NL,ES,PT,PL,IT,AT,DK,SE,GR,CZ
0,covid,covid,covide,covid,covid,covid,covid,covid,covid,covid,covid,covid,covid
1,corona,corona,corona,corona,corona,corona,korona,corona,corona,corona,corona,corona,corona
2,SARS-CoV-2,SARS-CoV-2,SRAS-CoV-2,SARS-CoV-2,SARS-CoV-2,SRA-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2
3,virus,Virus,virus,virus,virus,vírus,wirus,virus,Virus,virus,virus,ιός,virus
4,symptoms,Symptome,symptômes,symptomen,síntomas,sintomas,objawy,sintomi,Symptome,symptomer,symtom,συμπτώματα,příznaky


In [157]:
'''
while test_date_start <= test_date_end:
    if test_date_start not in data4['date']:
        print(test_date_start.strftime('%Y-%m-%d'))
        test_date_start += timedelta(days=1)
'''

'''
for i in data4['date']:
    if i == test_date:
        date_present = True
    else:
        pass
        
print(date_present)
'''

"\nfor i in data4['date']:\n    if i == test_date:\n        date_present = True\n    else:\n        pass\n        \nprint(date_present)\n"

In [64]:
import time
from datetime import date, timedelta

def g_trends_complete_timeframe(df, kw):
    # set start and end date for loop
    dt_s = df['date'][0]
    pos_end = len(df) - 1
    dt_e = df['date'][pos_end]
    
    # lists for missing days
    dates_missing = []
    trend_missing = []
    
    # loop to find missing dates in df
    while dt_s <= dt_e:
        date_check = False
        for d in df['date']:
            if d == dt_s:
                date_check = True
        # append missing date to lists
        if date_check == False:
            dates_missing.append(dt_s)
            trend_missing.append(np.NaN)
        dt_s += timedelta(days=1)
    
    # df for missing dates
    df_dates_missing = pd.DataFrame({'date': dates_missing, kw: trend_missing})
    df_completed = pd.concat([df, df_dates_missing], axis=0)
    df_completed['date'] = pd.to_datetime(df_completed['date'], errors='coerce')
    df_completed = df_completed.sort_values(by=['date'], ignore_index=True)
    
    return df_completed   


def g_trends_df_transform(df):
    # drop unwanted col
    df = df.drop(df.iloc[:, 1:], axis=1)
            
    # get date as col
    df.reset_index(inplace=True)
    df.columns = df.columns.str.replace('_unscaled', '')
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['date'] = df['date'].dt.date
    return df

def df_to_csv_format(df, name_cst, name_var, index=False):
    file_name = name_cst + '_' + name_var
    df.to_csv(f'{file_name}.csv', sep=',', index=index)
    return df


def g_trends_multilang(kw_dict, dt_start, dt_end, sleep_time=60):
    # df for all results
    trends_all = pd.DataFrame({})
    
    # loop through lang
    for geo in kw_dict.keys():
        # df for results of one loc
        search_trend = pd.DataFrame({})
        
        # loop through keywords
        for k in kw_dict[geo]:
            # get df with search trend for kw
            k_trend = dailydata.get_daily_data(k, int(dt_start.split('-')[0]), int(dt_start.split('-')[1]), int(dt_end.split('-')[0]), int(dt_end.split('-')[1]), geo=geo)
            time.sleep(sleep_time)
            
            # transform df
            k_trend = g_trends_df_transform(k_trend)
            
            # add missing dates
            k_trend_completed = g_trends_complete_timeframe(df=k_trend, kw=k)
            
            # add df with search trend for kw to df for results of one loc
            search_trend['date'] = k_trend_completed['date']
            search_trend[k] = k_trend_completed[k]
        
        # add loc to df 
        search_trend['location'] = geo
        # export df to csv
        search_details = dt_start + '_' + dt_end + '_' + geo
        df_to_csv_format(df=search_trend, name_cst='g_trend', name_var=search_details, index=False)
        
        # concat df with df holding all results
        if len(trends_all) > 0:
            search_trend.columns = trends_all.columns
        trends_all = pd.concat([trends_all, search_trend], axis=0)
    
    trends_all.reset_index(inplace=True)
    trends_all = trends_all.sort_values(by=['date'], ignore_index=True)
    
    return trends_all


In [39]:
test0_lang = {'GB': 'en', 'DK': 'da', 'GR': 'el'}
test0_kw = ['germany', 'Tesla', 'fever']

In [40]:
test0_dict, test0_df = deepl_translate_list(auth=deepl_auth_key, keywords=test0_kw, lang_out=test0_lang)

In [41]:
test0_df

,GB,DK,GR
0,germany,tyskland,Γερμανία
1,Tesla,Tesla,Tesla
2,fever,feber,πυρετός


In [57]:
test0_trends = g_trends_multilang(kw_dict=test0_dict, dt_start='2021-01', dt_end='2021-02')

germany:2021-01-01 2021-01-31
germany:2021-02-01 2021-02-28
         date  germany
0  2021-01-01       67
1  2021-01-02       64
2  2021-01-03       60
3  2021-01-04       95
4  2021-01-05       95
Tesla:2021-01-01 2021-01-31
Tesla:2021-02-01 2021-02-28
         date  Tesla
0  2021-01-01     41
1  2021-01-02     41
2  2021-01-03     34
3  2021-01-04     48
4  2021-01-05     51
fever:2021-01-01 2021-01-31
fever:2021-02-01 2021-02-28
         date  fever
0  2021-01-01     77
1  2021-01-02     80
2  2021-01-03     88
3  2021-01-04     72
4  2021-01-05     67
tyskland:2021-01-01 2021-01-31
tyskland:2021-02-01 2021-02-28
         date  tyskland
0  2021-01-01        85
1  2021-01-02        64
2  2021-01-03        77
3  2021-01-04        77
4  2021-01-05        59
Tesla:2021-01-01 2021-01-31
Tesla:2021-02-01 2021-02-28
         date  Tesla
0  2021-01-01     31
1  2021-01-02     45
2  2021-01-03     60
3  2021-01-04     72
4  2021-01-05     60
feber:2021-01-01 2021-01-31
feber:2021-02-01 2021-

In [62]:
test0_trends

,date,germany,Tesla,fever,location
0,2021-01-01,67.0,41.0,77.0,GB
1,2021-01-02,64.0,41.0,80.0,GB
2,2021-01-03,60.0,34.0,88.0,GB
3,2021-01-04,95.0,48.0,72.0,GB
4,2021-01-05,95.0,51.0,67.0,GB
5,2021-01-06,86.0,59.0,72.0,GB
6,2021-01-07,80.0,97.0,71.0,GB
7,2021-01-08,81.0,100.0,49.0,GB
8,2021-01-09,59.0,63.0,62.0,GB
9,2021-01-10,79.0,56.0,91.0,GB


In [235]:
# build payload
# pytrends.build_payload(kw_list=test0_kw, cat=0, timeframe='2020-01-02 2021-11-02', geo='GB')

In [203]:
date_t1 = datetime.datetime.strptime('2020-07-06', '%Y-%m-%d').date()
exists = date_t1 in data4['date']
exists

False

### 1.2.4 twitter keyword trends data

In [55]:
import tweepy as tw 

# get authentification for twitter API
with open('auth/tw_auth.txt') as t:
    cred = t.readlines()

tw_api_key = cred[1]
tw_api_key_sec = cred[3]
tw_token = cred[5]
tw_token_sec = cred[7]
#tw_bearer = cred[9]

# twitter API authentication
tw_auth = tw.AppAuthHandler(tw_api_key, tw_api_key_sec)
tw_auth.set_access_token(tw_token, tw_token_sec)

# authentication check
tw_api = tw.API(tw_auth)

try:
    tw_api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")



TweepyException: Expected token_type to equal "bearer", but got None instead